# Lesson 4: Link Function Exercises

## Imports and Setup

In [1]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm

from bambi import plots as bmb_plots
from matplotlib.lines import Line2D
from scipy import special, stats

rng_seed = np.random.RandomState(seed=12)

In [2]:
%matplotlib inline
plt.style.use("intuitivebayes.mplstyle")

## Exercise 1: Where do we see binary data?
Binary outcomes and probability are quite common in every day life. In this exercise you're asked to think about where you see these outcomes. Share your answers in the discourse

* Where typical you'd want to separate two groups?
* Where are places you'd like estimate probability of some quantity?

### Part 1: Separating groups
Where do you see yourself typically wanting to separate two outcomes? 

**Write answers here**

### Part 2: Estimating underlying probability

Where are places you'd like estimate probability of some quantity?

**Write answers here**

## Exercise 2: Estimating probability three ways

The goal is to learn there's not a unique right approach to model binary outcomes. On the contrary, there are many ways to do so, but they are all related. In this exercise we are going to generate binary observation data ourselves with known probability values, with a focus on the model.

We then are going to recover the latent probability using 2 different models implemented using our 2 favorite tools


1. Beta-Bernoulli regression
    * PyMC
    * Bambi
1. Logistic regression
    * PyMC
    * Bambi

So there will be a total of 4 models.

After completing this exercise once, you can go back, vary the parameters, and run the model inference again. This is what you could change:

1. The success probability of each Bernoulli distribution
2. The size of each example batch

### Data Generation

This code has been provided to you so you can generate the observations.

In [3]:
# These are the parameters were going to try and recover
prob_1 = 0.2
prob_2 = 0.8

# We can vary batch size as well
size_1 = 100
size_2 = 100

# Generate the random values and the batches identifiers
samples_1 = stats.bernoulli.rvs(p=prob_1, size=size_1, random_state=rng_seed)
samples_2 = stats.bernoulli.rvs(p=prob_2, size=size_2, random_state=rng_seed)
samples = np.concatenate([samples_1, samples_2])
batches = ["Batch 1"] * size_1 + ["Batch 2"] * size_2

Finally we put the values into a data frame and explore the first and last rows.

In [4]:
df_samples = pd.DataFrame({"result": samples, "batch": batches})
df_samples.head()

,result,batch
0,0,Batch 1
1,0,Batch 1
2,0,Batch 1
3,0,Batch 1
4,0,Batch 1


In [5]:
df_samples.tail()

,result,batch
195,1,Batch 2
196,1,Batch 2
197,0,Batch 2
198,0,Batch 2
199,1,Batch 2


Let's check the empirical success probabilities are close to the underlying true values.

In [6]:
df_samples.groupby("batch")["result"].mean()

batch
Batch 1    0.23
Batch 2    0.81
Name: result, dtype: float64

All good! Let's go!

### Part 1: Beta Bernoulli (or Binomial) model

Code a model in PyMC using a Beta prior. Feel free to use either a Bernoulli or Binomial likelihood. After coding the model verify that the posterior estimates the probabilities set above. After doing so 

1. Adjust the latent probability. Rerun the model and verify the results. Did your model recover those parameters?
2. Change the number of samples, Rerun the model and compare posteriors with a small number of samples vs a larger number

Bonus points for

* Only using one PyMC model context to estimate the latent probability of each group
* Coding this same model in Bambi

In [50]:
# Write code here

#### Bonus: Bambi Model
See if you can write the model in bambi as well


In [51]:
# Write code here

In [52]:
# Visualize posterior here

In [53]:
# Compare to pymc posterior here

#### Rethinking our choices

Think about the model you just coded.

1. Why did we use a Beta prior? 
2. How did it restrict the value of probability in a manner that we want per the definition of probability?
3. Now criticize the Beta prior we used.
    * Do you see any problem? Why?
    * Would you change something?
    * Share your answers in the discourse as well 😄

Write your answer below.

**Write answer here**

### Part 2: Logistic regression model

Now let's recover our parameters using our new tool, the generalized linear model and non-identiy link functions.

**Hint:** You'll need to use some transform that converts any values into the probability scale.

Once you've recovered the parameters answer these questions
1. What is the domain of our priors? Is is retricted to the range of probability, that is between 0 and 1?
2. What is the logistic function doing for us? How does it affect the possible output range?

#### The logistic regression model with PyMC

In [54]:
batch, batch_idx = np.unique(df_samples["batch"], return_inverse=True)
coords = {"batch": batch}

# Write model here

#### The logistic regression model with Bambi

Use Bambi to estimate the latent probability of each batch. We've provided some string encoding to help make your usage of Bambi even easier.

In [55]:
# Write bambi model here

## Exercise 3: Predicting Penguins
The Palmer Penguin dataset is beoming increasingly popular in the data community. We'll be using it for our next exercise.

In particular we want to see if we can classify Adelie Penguins from Chinstrap penguns using their physical features. In this case we'll be using their bill length ot predict the species.

### Data Loading
We've provided the code to load the data below. We're be using two columns.

* `bill_length_mm`: Bill length in millimeters
* `species`: A categorical column that indicates which species the observation is for

For reference the Palmer penguins dataset includes as number of numerical attributes of the penguin. A [full description](https://allisonhorst.github.io/palmerpenguins/) is available on the github page.

Let's now see if we can Bambi to predict which penguin is which based off of its `bill_length`.

In [20]:
penguins = pd.read_csv("data/penguins.csv")
missing_data = penguins.isnull()[
    ["bill_length_mm"]
].any(axis=1)

# Drop rows with any missing data
penguins = penguins.loc[~missing_data]

species_filter = penguins["species"].isin(["Adelie", "Chinstrap"])
penguins_filtered = penguins[species_filter]

penguins_observed = penguins_filtered.iloc[:-5, :]

In [21]:
penguins_observed.head()

,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,2,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,3,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
4,5,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
5,6,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007


### Part 1: Exploratory Data Analysis
Plot the species as a function of bill length. As statisticians we always want to do this to ensure we fully understand the data we are working with, verifying if it looks as expected, if there are any characteristics we weren't expecting, and ultimaetly to inform our modeling strategy,

In [56]:
# Write plotting code here

### Part 2: Fitting the Bambi Logistic Regression Model
Fit a a Bambi model that uses logistic regression to predict which penguin is which. **Hint**. To make this a GLM you'll need to use the family and/or link argument in the model call.

In [57]:
# In this answer Chinstrip ensure the penguin that is encoded 1 to match the plot above

### Part 3: Plotting the predictions
Use `bmb_plots` to overlay your previous EDA plot with the results from the logistic regression model. Take note of the HDI, where are we relatively certain of the penguin class, where are we less certain?

In [58]:
# Write plotting code here

### Part 2: Penguin Predictions
What if we now discover additional penguins with bill lengths of 30, 40, 50 respectively? What would we predict their species to be?

Using the logistic regression model we fit above, determine what the probability of each penguin will be.

**Hint** You will need to do some Inference data manipulation to get the final prediction values. This is due to the way Markov Chain Monte Carlo works. We've provided the code belopw to help.

`{az.InferenceData].posterior.species_mean.mean(dim=("chain","draw"))`

In [25]:
bill_lengths = [30, 40, 50]
new_obs = pd.DataFrame({"bill_length_mm":bill_lengths})

In [59]:
# Make predictions here

### Bonus: Plot the predictions

We can add the prediction to the plot as well!

In [61]:
# Plot predictions on previous inference plot

## Exercise 4: Will this mushroom kill you?

Mushrooms are delicious, except when they're poisonous and kill you. We'll be using the mushroom dataset provided by the [UCI machine learning repository](https://archive-beta.ics.uci.edu/dataset/73/mushroom). Our goal in short is to predict


We want to understand which qualities predict mushroom toxicity. Bambi makes exploratory model building such as this quick, with fast model definitions and quick feedback loops. 

The full definition of the dataset can be found in the file [agaricus-lepiota.names](data/agaricus-lepiota.names). We've pulled the attribute names section from the data file.

```
7. Attribute Information: (classes: edible=e, poisonous=p)
     1. cap-shape:                bell=b,conical=c,convex=x,flat=f,
                                  knobbed=k,sunken=s
     2. cap-surface:              fibrous=f,grooves=g,scaly=y,smooth=s
     3. cap-color:                brown=n,buff=b,cinnamon=c,gray=g,green=r,
                                  pink=p,purple=u,red=e,white=w,yellow=y
     4. bruises?:                 bruises=t,no=f
     5. odor:                     almond=a,anise=l,creosote=c,fishy=y,foul=f,
                                  musty=m,none=n,pungent=p,spicy=s
     6. gill-attachment:          attached=a,descending=d,free=f,notched=n
     7. gill-spacing:             close=c,crowded=w,distant=d
     8. gill-size:                broad=b,narrow=n
     9. gill-color:               black=k,brown=n,buff=b,chocolate=h,gray=g,
                                  green=r,orange=o,pink=p,purple=u,red=e,
                                  white=w,yellow=y
    10. stalk-shape:              enlarging=e,tapering=t
    11. stalk-root:               bulbous=b,club=c,cup=u,equal=e,
                                  rhizomorphs=z,rooted=r,missing=?
    12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
    13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
    14. stalk-color-above-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o,
                                  pink=p,red=e,white=w,yellow=y
    15. stalk-color-below-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o,
                                  pink=p,red=e,white=w,yellow=y
    16. veil-type:                partial=p,universal=u
    17. veil-color:               brown=n,orange=o,white=w,yellow=y
    18. ring-number:              none=n,one=o,two=t
    19. ring-type:                cobwebby=c,evanescent=e,flaring=f,large=l,
                                  none=n,pendant=p,sheathing=s,zone=z
    20. spore-print-color:        black=k,brown=n,buff=b,chocolate=h,green=r,
                                  orange=o,purple=u,white=w,yellow=y
    21. population:               abundant=a,clustered=c,numerous=n,
                                  scattered=s,several=v,solitary=y
    22. habitat:                  grasses=g,leaves=l,meadows=m,paths=p,
                                  urban=u,waste=w,woods=d
```

With these attributes see what you can find in the dataset.See what you can find.  Share your results on Discourse. This is an open ended question and no right or wrong answer.

### Data Loading

In [30]:
mushrooms = pd.read_csv("data/mushrooms.csv")
mushrooms.columns = mushrooms.columns.str.replace(pat="-", repl="_")

mushrooms.head()

,class,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


### Part 1: Exploratory Data Analysis

We first check the number of occurences of both types of mushrooms, poisonous and edible. In this dataset the number of rows looks relatively balanced.

In [31]:
# Perform EDA of mushroom dataset

e    4208
p    3916
Name: class, dtype: int64

### Part 2: Logistic Regression Modeling

Based off of EDA lets try modeling gill-size, ring-type, and odor. from that we can get a sense of which characteristics help us best estimate which mushroom is edible versus those that are not.

Before we run our sampler we'll recode the categorical indicators so its easier to understand their physical meaning in the subsequent plots.

In [33]:
mappings = dict(
  gill_size = dict(b='broad',n='narrow'),
  ring_type = dict(c='cobwebby',e='evanescent',f='flaring', l='large', n='none', p='pendant',s='sheathing', z='zone='),
  odor= dict(a='almond',l='anise',c='creosote',y='fishy',f='foul',m='musty',n='none',p='pungent',s='spicy'))

In [62]:
# Write regression using bambi to predict mushroom toxicity
# You may use the remapping above if you'd like the output label names to be nice

### Bonus: Create a ridgeplot

Tables can be challenging to understand at a glance. We can also create a ridge plot to get a sense of the range and distribution of the posterior plot. Let's also add a vertical dashed line at 0 to make it easier to see what provides

In [63]:
# Add ridgeplot here

## Exercise 5: Complete separability

Let's now try something else with our mushroom dataset. Imagine a case where we pick with a certain gill attachment, but we observe they're all poisonous. This is a complete class separation.

In exercise you will be comparing maximum likelihood estimators with Bayesian estimators in a specific scenario, where one covariate has not observed both classes.



### Data Set Construction
Let's create that dataset ourselves. We're doing so, so we can focus on the mathematical idea here rather than a new dataset.

In [39]:
# All rows where gill attachment is a and the mushrooms are poisonous
poisonous_a_index =  mushrooms.query("gill_attachment == 'a' and `class` == 'p'").index
subset_mushrooms = mushrooms[~mushrooms.index.isin(poisonous_a_index)][["gill_attachment", "ring_number", "class"]]

Great now we have data set where every mushroom of gill_attachment `a` is poisonous. 

### EDA 

Now because we constructed this dataset the results of this EDA are not surprising to us. However because we're good statisticians we always want to make sure we don't skip this important step. 

In [40]:
# Verify that gill_attachemnt a only has edible mushrooms
subset_mushrooms.groupby(["gill_attachment", "ring_number", "class"]).size()

gill_attachment  ring_number  class
a                o            e         192
f                n            p          18
                 o            e        3488
                              p        3808
                 t            e         528
                              p          72
dtype: int64

### Part 1: Maximum Likelihood Estimation
Here we've setup the statsmodels regression for you. Run the model and see what the results are.
You may need to install statsmodel into the environment

In [41]:
# Statsmodel requires the output column to be binary so we need to do this conversion ourselves
# Bambi handles this for us automatically!
subset_mushrooms["class_int"] = subset_mushrooms["class"].replace({"e":0, "p":1})

In [64]:
import statsmodels.formula.api as smf

# log_reg = smf.logit("class_int ~ ring_number + gill_attachment", data=subset_mushrooms).fit()

In [65]:
# Inspect summary
# log_reg.summary()

### Part 2: Why does the model fail?

Explain why the model fails

### Part 3: Bayesian Regression
Now lets try Bayesian regression. Before fitting the model think through one question. In Bayesian models what's one extra piece of information that is not used in frequentist or maximum likelihood methods?

In [66]:
# Write the equivalent bambi model here